In [1]:
import pandas as pd
from IPython.display import display
import mysql.connector
import os
from dotenv import load_dotenv

# Extract Data

In [2]:
def ingestion(table):
    load_dotenv()
    connection = mysql.connector.connect(
    host=os.getenv('host'),
    user='root',
    password=os.getenv('password'),
    database='capstone5'
    )
    query = f"SELECT * FROM {table}"  
    df = pd.read_sql(query, connection)
    connection.close()
    return df

In [4]:
df_applications = ingestion('applications')
df_articles = ingestion('articles')
df_comments = ingestion('comments')
df_donations = ingestion('donations')
df_donation_comments = ingestion('donation_comments')
df_fundraising_categories = ingestion('fundraising_categories')
df_fundraisings = ingestion('fundraisings')

df_customers = ingestion('users')
df_likes_comments = ingestion('likes_comments')
df_organizations = ingestion('organizations')
df_testimoni_volunteers = ingestion('testimoni_volunteers')
df_bookmark_fundraising = ingestion('user_bookmark_fundraising')
df_bookmark_volunteer = ingestion('user_bookmark_volunteer_vacancies')
df_volunteer_vacancies = ingestion('volunteers')

C:\Users\SINTA\AppData\Local\Temp\ipykernel_23344\3238963636.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)
C:\Users\SINTA\AppData\Local\Temp\ipykernel_23344\3238963636.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)
C:\Users\SINTA\AppData\Local\Temp\ipykernel_23344\3238963636.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)


In [ ]:
display(df_applications)
display(df_articles)
display(df_comments)
display(df_donations)
display(df_donation_comments)
display(df_fundraising_categories)
display(df_fundraisings)
display(df_customers)
display(df_likes_comments)
display(df_testimoni_volunteers)
display(df_volunteer_vacancies)

# Transform

## Tabel Fakta

### 1. FactDonationTransaction

In [ ]:
# Buat Struktur kolom df_fact_donation_transaction
columns = ['id', 'donation_id', 'fundraising_id', 'customer_id', 'nominal', 'goal_amount', 'fundraising_categories_id', 'organization_id', 'donation_created_at']
df_fact_donation_transaction = pd.DataFrame(columns=columns)

In [ ]:
# mengambil data yang sukses di df_donation 
df_donation_success = df_donations.loc[df_donations['status'] == 'success']
df_donation_success = df_donation_success.reset_index(drop=True)

In [ ]:
# mengisi data pada dari kolom df_donation
df_fact_donation_transaction['id'] = range(1, len(df_donation_success) + 1)
df_fact_donation_transaction['donation_id'] = df_donation_success['id']
df_fact_donation_transaction['fundraising_id'] = df_donation_success['fundraisings_id']
df_fact_donation_transaction['customer_id'] = df_donation_success['customer_id']
df_fact_donation_transaction['nominal'] = df_donation_success['nominal']
df_fact_donation_transaction['donation_created_at'] = df_donation_success['created_at']

In [ ]:
# merge df_fundraising
df_merge_fact_fundraising = pd.merge(df_fact_donation_transaction, df_fundraisings, left_on='fundraising_id', right_on='id', how='left')

# inisialisasi data di df_fact_donation_transaction
df_fact_donation_transaction['goal_amount'] = df_merge_fact_fundraising['goal_amount_y']
df_fact_donation_transaction['fundraising_categories_id'] = df_merge_fact_fundraising['fundraising_categories_id_y']
df_fact_donation_transaction['organization_id'] = df_merge_fact_fundraising['organization_id_y']
df_fact_donation_transaction

### 2. Fact Volunteer Applications

In [ ]:
# Buat Struktur kolom df_fact_volunteer_applications
columns = ['id', 'application_id', 'volunteer_vacancies_id', 'customer_id', 'organization_id', 'application_created_at']
df_fact_volunteer_applications = pd.DataFrame(columns=columns)

In [ ]:
# mengisi data pada dari kolom df_application
df_fact_volunteer_applications['id'] = range(1, len(df_applications) + 1)
df_fact_volunteer_applications['application_id'] = df_applications['id']
df_fact_volunteer_applications['volunteer_vacancies_id'] = df_applications['vacancy_id']
df_fact_volunteer_applications['customer_id'] = df_applications['customer_id']
df_fact_volunteer_applications['application_created_at'] = df_applications['create_at']
df_fact_volunteer_applications

In [ ]:
# merge df_fundraising
df_merge_fact_volunteer = pd.merge(df_fact_volunteer_applications, df_volunteer_vacancies, left_on='volunteer_vacancies_id', right_on='id', how='left')

# inisialisasi data di df_fact_volunteer_applications
df_fact_volunteer_applications['organization_id'] = df_merge_fact_volunteer['organization_id_y']
df_fact_volunteer_applications

### 3. Fact Volunteer Testimoni

In [ ]:
# Buat Struktur kolom df_fact_volunteer_testimoni
columns = ['id', 'customer_id', 'volunteer_vacancies_id', 'testimoni_volunteer_id', 'rating']
df_fact_volunteer_testimoni = pd.DataFrame(columns=columns)

In [ ]:
# mengisi data pada dari kolom df_application
df_fact_volunteer_testimoni['id'] = range(1, len(df_applications) + 1)
df_fact_volunteer_testimoni['customer_id'] = df_testimoni_volunteers['customer_id']
df_fact_volunteer_testimoni['volunteer_vacancies_id'] = df_testimoni_volunteers['volunteer_id']
df_fact_volunteer_testimoni['testimoni_volunteer_id'] = df_testimoni_volunteers['id']
df_fact_volunteer_testimoni['rating'] = df_testimoni_volunteers['rating']
df_fact_volunteer_applications

### 4. Fact Article Popular

In [ ]:
# Buat Struktur kolom df_fact_articel_popular
columns = ['id', 'article_id', 'comment_id', 'customer_id', 'total_like', 'comment_created_at']
df_fact_article_popular = pd.DataFrame(columns=columns)

In [ ]:
# mengisi data pada dari kolom df_comment
df_fact_article_popular['id'] = range(1, len(df_comments) + 1)
df_fact_article_popular['article_id'] = df_comments['article_id']
df_fact_article_popular['comment_id'] = df_comments['id']
df_fact_article_popular['customer_id'] = df_comments['customer_id']
df_fact_article_popular['total_like'] = df_comments['total_like']
df_fact_article_popular['comment_created_at'] = df_comments['created_at']
df_fact_article_popular

### 5. Fact Bookmark Fundraising

In [ ]:
df_fact_bookmark_fundraising = df_bookmark_fundraising.drop(['deleted_at', 'updated_at'], axis=1)
df_fact_bookmark_fundraising = df_fact_bookmark_fundraising.rename(columns={'id':'bookmark_id'})
df_fact_bookmark_fundraising['id'] = range(1, len(df_fact_bookmark_fundraising) + 1)
df_fact_bookmark_fundraising.insert(0, 'id', df_fact_bookmark_fundraising.pop('id'))
df_fact_bookmark_fundraising

### 6. Fact Bookmark Volunteer Vacancies

In [ ]:
df_fact_bookmark_volunteer_vacancies = df_bookmark_volunteer.drop(['deleted_at', 'updated_at'], axis=1)
df_fact_bookmark_volunteer_vacancies = df_fact_bookmark_volunteer_vacancies.rename(columns={'id':'bookmark_id'})
df_fact_bookmark_volunteer_vacancies['id'] = range(1, len(df_fact_bookmark_volunteer_vacancies) + 1)
df_fact_bookmark_volunteer_vacancies.insert(0, 'id', df_fact_bookmark_volunteer_vacancies.pop('id'))
df_fact_bookmark_volunteer_vacancies

## Tabel Dimensi

In [ ]:
dim_fundraisings = df_fundraisings.drop(['fundraising_categories_id','organization_id','updated_at'], axis=1)

dim_fundraising_categories = df_fundraising_categories[['id','category_name','update_at']]
dim_donation = df_donations.drop(['fundraisings_id', 'customer_id','updated_at'], axis=1)
dim_organization = df_organizations
dim_customer = df_customers
dim_volunteer_applictaion = df_applications.drop(['customer_id','vacancy_id','update_at'], axis=1)
dim_volunteer_vacancies = df_volunteer_vacancies.drop(['organization_id','updated_at'], axis=1)
dim_testimoni_volunteer = df_testimoni_volunteers.drop(['customer_id','volunteer_id','updated_at'], axis=1)
dim_article = df_articles
dim_comment = df_comments.drop(['customer_id','article_id','updated_at'], axis=1)
dim_bookmark_fundraising = df_bookmark_fundraising.drop(['fundraising_id','customer_id','updated_at'], axis=1)
dim_bookmark_volunter_vacancies = df_bookmark_volunteer.drop(['volunteer_vacancies_id','customer_id','updated_at'], axis=1)


# Load

In [ ]:
df_fact_donation_transaction.to_csv("Data_Loaded/Fact/fact_donation_transaction.csv", index=False)
df_fact_volunteer_applications.to_csv("Data_Loaded/Fact/fact_volunteer_applications.csv", index=False)
df_fact_volunteer_testimoni.to_csv("Data_Loaded/Fact/fact_volunteer_testimoni.csv", index=False)
df_fact_article_popular.to_csv("Data_Loaded/Fact/fact_article_popular.csv", index=False)


